## Statoil Iceberg Classification

### Ensemble learning
### Stacking several CNN prediction result for better accuracy

In [1]:
import os
import numpy as np 
import pandas as pd 
from subprocess import check_output
print(check_output(["ls", "."]).decode("utf8"))

Iceberg_py3_keras.ipynb
Iceberg_py3_keras_angle.ipynb
README.md
bad_sub
ensemble.ipynb
stack_median.csv
stack_median_3.csv
stack_minmax_median.csv
sub.csv
submission
test.json
train.json



In [2]:
sub_path = "./submission"
all_files = os.listdir(sub_path)

outs = [pd.read_csv(os.path.join(sub_path, f), index_col=0) for f in all_files]
concat_sub = pd.concat(outs, axis=1)
cols = list(map(lambda x: "is_iceberg_" + str(x), range(len(concat_sub.columns))))
concat_sub.columns = cols
concat_sub.reset_index(inplace=True)
concat_sub.head()

,id,is_iceberg_0,is_iceberg_1,is_iceberg_2,is_iceberg_3
0,5941774d,2.301870e-03,0.036318,0.128144,0.000943
1,4023181e,9.994224e-01,0.995405,0.352253,0.946086
2,b20200e4,7.558256e-22,0.004805,0.007293,0.006515
3,e7f018bb,9.972625e-01,0.997890,0.993945,0.999929
4,4371c8c3,1.000000e+00,0.994407,0.652634,0.850582


In [3]:
concat_sub.corr()

,is_iceberg_0,is_iceberg_1,is_iceberg_2,is_iceberg_3
is_iceberg_0,1.000000,0.702787,0.532062,0.655352
is_iceberg_1,0.702787,1.000000,0.738956,0.790478
is_iceberg_2,0.532062,0.738956,1.000000,0.699070
is_iceberg_3,0.655352,0.790478,0.699070,1.000000


In [5]:
# get the data fields ready for stacking
concat_sub['is_iceberg_max'] = concat_sub.iloc[:, 1:5].max(axis=1)
concat_sub['is_iceberg_min'] = concat_sub.iloc[:, 1:5].min(axis=1)
concat_sub['is_iceberg_mean'] = concat_sub.iloc[:, 1:5].mean(axis=1)
concat_sub['is_iceberg_median'] = concat_sub.iloc[:, 1:5].median(axis=1)

In [6]:
# set up cutoff threshold for lower and upper bounds, easy to twist 
cutoff_lo = 0.8
cutoff_hi = 0.2

### Median Stacking

In [7]:
concat_sub['is_iceberg'] = concat_sub['is_iceberg_median']
concat_sub[['id', 'is_iceberg']].to_csv('stack_median_4.csv', 
                                        index=False, float_format='%.6f')

### MinMax Median Stacking

In [8]:
concat_sub['is_iceberg'] = np.where(np.all(concat_sub.iloc[:,1:5] > cutoff_lo, axis=1), 
                                    concat_sub['is_iceberg_max'], 
                                    np.where(np.all(concat_sub.iloc[:,1:5] < cutoff_hi, axis=1),
                                             concat_sub['is_iceberg_min'], 
                                             concat_sub['is_iceberg_median']))
concat_sub[['id', 'is_iceberg']].to_csv('stack_minmax_median_2.csv', 
                                        index=False, float_format='%.6f')